# Geospatial

การแสดงข้อมูลทางภูมิศาสตร์ ( Geospatial ) แบบ Basic เพื่อแยกว่าภาพใดเป็น Dog ภาพใดเป็น Cat ซึ่งเป็นพื้นฐานของการทำ Machine Learning และ Deep Learning โดยเราจะใช้ Convolutional Neural Network ( CNN ) ด้วย Keras รันบน Tensorflow

In [1]:
! pip install geopandas
! pip install folium
! pip install kaggle

In [2]:
% cd /root
! mkdir -p ~/.kaggle

/root


In [3]:
% cd .kaggle

/root/.kaggle


## Kaggle

ทำการดาวน์โหลดไฟล์ Dataset ที่จะใช้ในการ Plot Graph จาก Kaggle

In [0]:
! touch kaggle.json
! chmod 600 kaggle.json

In [0]:
f = open("kaggle.json","w+")
f.write('{"username":"natthasath","key":"7a892a2a13cef788cb2454201e6708dd"}')
f.close() 

In [6]:
! kaggle --help

usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...

optional arguments:
  -h, --help            show this help message and exit
  -v, --version         show program's version number and exit

commands:
  {competitions,c,datasets,d,kernels,k,config}
                        Use one of:
                        competitions {list, files, download, submit, submissions, leaderboard}
                        datasets {list, files, download, create, version, init, metadata, status}
                        config {view, set, unset}
    competitions        Commands related to Kaggle competitions
    datasets            Commands related to Kaggle datasets
    kernels             Commands related to Kaggle kernels
    config              Configuration settings


In [7]:
% cd /content

/content


In [8]:
#! kaggle competitions download -c 'alexisbcook/geospatial-learn-course-data'
! kaggle datasets download -d 'alexisbcook/geospatial-learn-course-data'

geospatial-learn-course-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
! unzip /content/geospatial-learn-course-data.zip

Archive:  /content/geospatial-learn-course-data.zip
replace CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.cpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.cpg  
  inflating: CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.dbf  
  inflating: CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.prj  
  inflating: CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.shp  
  inflating: CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.shx  
  inflating: CA_county_high_earners.csv  
  inflating: CA_county_median_age.csv  
  inflating: CA_county_population.csv  
  inflating: DEC_lands/DEC_lands/DEC_Lands.html  
  inflating: DEC_lands/DEC_lands/DEC_lands.cpg  
  inflating: DEC_lands/DEC_lands/DEC_lands.dbf  
  inflating: DEC_lands/DEC_lands/DEC_lands.prj  
  inflating: DEC_lands/DEC_lands/DEC_lands.sbn  
  inflating: DEC_lands/DEC_lands/DEC_lands.sbx  
  inf

## Interactive Map

In [0]:
import pandas as pd
import geopandas as gpd
import math
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

In [11]:
# Create a map
m_1 = folium.Map(location=[42.32,-71.0589], tiles='openstreetmap', zoom_start=10)

# Display the map
m_1

In [12]:
# Load the data
crimes = pd.read_csv("/content/crimes-in-boston/crimes-in-boston/crime.csv", encoding='latin-1')

# Drop rows with missing locations
crimes.dropna(subset=['Lat', 'Long', 'DISTRICT'], inplace=True)

# Focus on major crimes in 2018
crimes = crimes[crimes.OFFENSE_CODE_GROUP.isin([
    'Larceny', 'Auto Theft', 'Robbery', 'Larceny From Motor Vehicle', 'Residential Burglary',
    'Simple Assault', 'Harassment', 'Ballistics', 'Aggravated Assault', 'Other Burglary', 
    'Arson', 'Commercial Burglary', 'HOME INVASION', 'Homicide', 'Criminal Harassment', 
    'Manslaughter'])]
crimes = crimes[crimes.YEAR>=2018]

# Print the first five rows of the table
crimes.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
6,I182070933,724,Auto Theft,AUTO THEFT,B2,330,NaN,2018-09-03 21:25:00,2018,9,Monday,21,Part One,NORMANDY ST,42.306072,-71.082733,"(42.30607218, -71.08273260)"
8,I182070931,301,Robbery,ROBBERY - STREET,C6,177,NaN,2018-09-03 20:48:00,2018,9,Monday,20,Part One,MASSACHUSETTS AVE,42.331521,-71.070853,"(42.33152148, -71.07085307)"
19,I182070915,614,Larceny From Motor Vehicle,LARCENY THEFT FROM MV - NON-ACCESSORY,B2,181,NaN,2018-09-02 18:00:00,2018,9,Sunday,18,Part One,SHIRLEY ST,42.325695,-71.068168,"(42.32569490, -71.06816778)"
24,I182070908,522,Residential Burglary,BURGLARY - RESIDENTIAL - NO FORCE,B2,911,NaN,2018-09-03 18:38:00,2018,9,Monday,18,Part One,ANNUNCIATION RD,42.335062,-71.093168,"(42.33506218, -71.09316781)"


## Point Marker

In [0]:
daytime_robberies = crimes[((crimes.OFFENSE_CODE_GROUP == 'Robbery') & \
                            (crimes.HOUR.isin(range(9,18))))]

In [14]:
# Create a map
m_2 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
for idx, row in daytime_robberies.iterrows():
    Marker([row['Lat'], row['Long']]).add_to(m_2)

# Display the map
m_2

## Point Marker Cluster

In [15]:
# Create the map
m_3 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
mc = MarkerCluster()
for idx, row in daytime_robberies.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(Marker([row['Lat'], row['Long']]))
m_3.add_child(mc)

# Display the map
m_3

## Bubble Map

In [16]:
# Create a base map
m_4 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

def color_producer(val):
    if val <= 12:
        return 'forestgreen'
    else:
        return 'darkred'

# Add a bubble map to the base map
for i in range(0,len(daytime_robberies)):
    Circle(
        location=[daytime_robberies.iloc[i]['Lat'], daytime_robberies.iloc[i]['Long']],
        radius=20,
        color=color_producer(daytime_robberies.iloc[i]['HOUR'])).add_to(m_4)

# Display the map
m_4

## Heat Map

In [17]:
# Create a base map
m_5 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)

# Add a heatmap to the base map
HeatMap(data=crimes[['Lat', 'Long']], radius=10).add_to(m_5)

# Display the map
m_5

## Choropleth Map

In [18]:
# GeoDataFrame with geographical boundaries of Boston police districts
districts_full = gpd.read_file('/content/Police_Districts/Police_Districts/Police_Districts.shp')
districts = districts_full[["DISTRICT", "geometry"]].set_index("DISTRICT")
districts.head()

,geometry
DISTRICT,
A15,"MULTIPOLYGON (((-71.07416 42.39051, -71.07415 ..."
A7,"MULTIPOLYGON (((-70.99644 42.39557, -70.99644 ..."
A1,"POLYGON ((-71.05200 42.36884, -71.05169 42.368..."
C6,"POLYGON ((-71.04406 42.35403, -71.04412 42.353..."
D4,"POLYGON ((-71.07416 42.35724, -71.07359 42.357..."


In [19]:
# Number of crimes in each police district
plot_dict = crimes.DISTRICT.value_counts()
plot_dict.head()

D4     2885
B2     2231
A1     2130
C11    1899
B3     1421
Name: DISTRICT, dtype: int64

In [20]:
# Create a base map
m_6 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)

# Add a choropleth map to the base map
Choropleth(geo_data=districts.__geo_interface__, 
           data=plot_dict, 
           key_on="feature.id", 
           fill_color='YlGnBu', 
           legend_name='Major criminal incidents (Jan-Aug 2018)'
          ).add_to(m_6)

# Display the map
m_6

Output hidden; open in https://colab.research.google.com to view.

https://www.kaggle.com/alexisbcook/geospatial-learn-course-data

https://www.kaggle.com/learn/geospatial-analysis